In [12]:
import re
import pandas as pd

In [13]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
t1 = pd.read_csv(fn, header=0)

In [14]:
from tidylib import tidy_document
import html2text

In [15]:
def cleanup(txt):
    doc, errors = tidy_document(str(txt))
    doc = html2text.html2text(doc)
    return doc.replace('*','').replace('_','').lower().strip()
    
def add_clean_text(row):
    row['scrape'] = cleanup(row.AdditionalDescription)
    return row

In [16]:
t1_fix = t1.apply(add_clean_text,1)

In [17]:
# I only care about these columns for now
cols = ['RequestID', 
 'ConfirmationNumber',
 'AgencyCode',
 'AgencyName',
 'AgencyDivision',
 'SectionID',
 'SectionName',
 'scrape']
fixed = t1_fix[cols]

# top 10: breakdown of ads by agencies
fixed['AgencyName'].value_counts()[:10]

Mayor's Office of Contract Services       67
Community Boards                          36
Citywide Administrative Services          35
Housing Preservation and Development      18
Landmarks Preservation Commission         18
Human Resources Administration            17
Transportation                            16
Health and Mental Hygiene                 15
Administration for Children's Services    14
City Planning                             13
dtype: int64

In [18]:
# Focus on Mayor's Office of Contract of Services
# because there seems to be a fixed format. Can Regex be useful?
#
# Example entry:
#     public notice is hereby given that the 
#      franchise and concession review committee will hold a 
#      public meeting on wednesday, october 8, 2014 at 2:30 p.m., 
#       at 22 reade street, spector hall, borough of manhattan. 
#

mocs = fixed['AgencyName'] == "Mayor's Office of Contract Services"
mocs_ads = fixed[mocs][['RequestID', 'scrape']]

# build a regex
rex_time = 'public notice is hereby given that the\s+franchise and concession review committee will hold a\s+public meeting on\s+(?P<day>[^\s,]+)[\s,]\s+(?P<month>[^\s,]+)\s+(?P<date>\d+)[,\s]+(?P<year>\d+)\s+(at)?\s*(?P<hour>\d+):(?P<minute>\d+)\s+(?P<tod>\w+\.?\w+\.?)'
rex_time = re.compile(rex_time, re.IGNORECASE|re.DOTALL|re.MULTILINE)



In [19]:
# Iterate over the data and see if we can scrape
#
moc_ads = mocs_ads['scrape'].values[0]
for ad in moc_ads:
    # remove multiple spaces and newlines
    ad = re.sub('[ \n] +', ' ', moc_ads)
    # extract ...
    print(rex_time.findall(ad))


[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october', '8', '2014', 'at', '2', '30', 'p.m.')]
[('wednesday', 'october',